In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import yaml
import dask.array as da
from ag3 import release_data
v3_release = release_data()

# supplementary table - sample level
### get meta

In [2]:
#samplesets
samplesets = v3_release.all_wild_sample_sets
df_meta = v3_release.load_sample_set_metadata(samplesets, True)
#fixes
df_meta.country = df_meta.country.str.replace("Gambia, The", "The Gambia").str.replace("Equatorial Guinea", "Bioko")

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [3]:
df_supp = df_meta.reset_index()
df_supp.columns

Index(['sample_id', 'partner_sample_id', 'contributor', 'country', 'location',
       'year', 'month', 'latitude', 'longitude', 'sex_call', 'sample_set',
       'aim_fraction_colu', 'aim_fraction_arab', 'species_gambcolu_arabiensis',
       'species_gambiae_coluzzii', 'is_arabiensis', 'is_gamb_colu',
       'is_gambiae', 'is_coluzzii'],
      dtype='object')

In [4]:
df_supp = df_supp[['sample_id', 'partner_sample_id', 'sample_set', 'country', 'location',
       'year', 'month', 'latitude', 'longitude', 'sex_call', 'aim_fraction_colu', 'aim_fraction_arab', 'species_gambcolu_arabiensis',
       'species_gambiae_coluzzii', 'is_arabiensis', 'is_gamb_colu',
       'is_gambiae', 'is_coluzzii']]

In [5]:
df_supp.rename(columns={'location':'site'}, inplace=True)

In [6]:
#round
df_supp = df_supp.round(3)
#export
df_supp.to_csv("../content/tables/supplementary_sampling_location_table.csv", index=False)

# main text table - site >10 level
### fix meta

In [7]:
#consolidate all Mayotte sites into one (so we get a pie chart)
df_meta.loc[(df_meta.country == 'Mayotte'),'latitude'] = df_meta[df_meta.country == 'Mayotte'].latitude.mean()
df_meta.loc[(df_meta.country == 'Mayotte'),'longitude'] = df_meta[df_meta.country == 'Mayotte'].longitude.mean()
#counts by sites
df_site_counts = df_meta.groupby(['latitude', 'longitude', 'country']).size().reset_index(name='counts')
df_site_counts['site'] = range(len(df_site_counts))
#>= 10 sites 
df_10 = df_site_counts[df_site_counts.counts >= 10].copy()
len(df_10),len(df_10.latitude.unique()),len(df_10.longitude.unique())

(43, 43, 42)

In [8]:
#new full meta but only containing >=10 sites (to preserve all species columns)
df_10_meta = df_meta.merge(df_10[['longitude','latitude', 'site']])
len(df_10_meta.latitude.unique()),len(df_10_meta.longitude.unique())

(43, 42)

### get crosstabbing

In [9]:
arab_call = pd.crosstab(df_10_meta.site, df_10_meta.species_gambcolu_arabiensis)
colu_call = pd.crosstab(df_10_meta.site, df_10_meta.species_gambiae_coluzzii)
colu_call.rename({"intermediate": "gambiae/coluzzii"}, axis=1, inplace=True)
colu_call.head()

species_gambiae_coluzzii,coluzzii,gambiae,gambiae/coluzzii
site,,,
0,0,74,0
2,0,23,0
3,81,0,0
4,0,36,6
5,0,28,45


In [10]:
sex_call = pd.crosstab(df_10_meta.site, df_10_meta.sex_call)
sex_call.rename({"F": "female", "M": "male", "UKN": "sex_unknown"}, axis=1, inplace=True)
sex_call.head()

sex_call,female,male,sex_unknown
site,,,
0,74,0,0
1,41,0,0
2,11,12,0
3,77,4,0
4,43,0,0


### get building

In [11]:
#reindex and join
df_10 =  df_10.set_index('site')
df_10_final = df_10.join([arab_call, colu_call, sex_call])

In [12]:
#add locations - must be a better way of doing this using merge maybe?
site = []
for _, row in df_10_final.iterrows():
    all_site = df_meta[(df_meta.latitude == row.latitude) & (df_meta.longitude == row.longitude)].copy()
    if all_site.location.unique()[0] != 'Mtsamboro Forest Reserve':
        site.append(all_site.location.unique()[0])
    else:
        #fix_mayotte
        site.append('Mayotte')
        
df_10_final['region'] = site

In [13]:
#remove nans, fix types
df_10_final = df_10_final.fillna(value=0).astype({'coluzzii':int, 'gambiae':int, 'gambiae/coluzzii':int})

In [14]:
df_10_final = df_10_final[['country', 'region', 'latitude', 'longitude', 'counts', 'female', 'male', 
                           'sex_unknown', 'arabiensis', 'intermediate', 'coluzzii', 'gambiae', 'gambiae/coluzzii']]

#fix names
df_10_final.rename(columns={"region": "site", "sex_unknown": "sex unknown", "intermediate": "arabiensis x   gambiae", 
                            "gambiae/coluzzii": "gambiae x coluzzii"}, inplace=True)
#sort
df_10_final.sort_values(by=['country'], inplace=True)
#round
df_10_final = df_10_final.round(3)
#export
df_10_final.to_csv("../content/tables/main_sampling_location_table.csv", index=False)